# Counting Squarefree numbers

A positive integer that is not divisible by the square of any prime is called **squarefree**. For example, $5$ is squarefree, but $20$ is not, since $2^2 \mid 20$.

Let $Q(x)$ denote the number of squarefree integers not exceeding $x$.

In this notebook, we will cover methods to count squarefree numbers.

In [10]:
import numpy as np

from typing import Dict, List

from core import integer_sqrt

## Brute force counting

It is simple to determine whether an integer is squarefree based on its prime factorization. We can obtain the factorizations of all positive integers below a bound by a simple sieve.

In [11]:
def factorizations(n: int) -> Dict[int, Dict[int, int]]:
    """Returns dict containing factorizations of integers k, 2 <= k <= n.
    
    Parameters:
        n: int
    
    Returns:
        Dict[int, Dict[int, int]]: Map containing factorizations for each
        integer 2 <= k <= n. Each factorization is a map prime -> exponent.
    """
    block = list(range(n))
    factors = {k: {} for k in range(1, n)}
    factors[1] = {1: 1}
    
    for d in range(2, n):
        if block[d] == 1:
            continue
            
        for multiple in range(d, n, d):
            factors[multiple][d] = 0
            
            while block[multiple] % d == 0:
                block[multiple] //= d
                factors[multiple][d] += 1
    
    return factors


# Simple sanity test on small values
for (k, factorization) in factorizations(1000).items():
    product = 1
    for (p, e) in factorization.items():
        product *= p**e
    assert k == product

In [12]:
def divisors(n: int) -> Dict[int, Dict[int, int]]:
    """Returns dict containing factorizations of integers k, 2 <= k <= n.
    
    Parameters:
        n: int
    
    Returns:
        Dict[int, Dict[int, int]]: Map containing factorizations for each
        integer 2 <= k <= n. Each factorization is a map prime -> exponent.
    """
    divisors = {k: [1] for k in range(1, n)}
    
    for d in range(2, n):            
        for multiple in range(d, n, d):
            divisors[multiple].append(d)
    
    return divisors

In [13]:
# Prime factorizations of integers < 20, given as prime: exponent dicts
factorizations(20)

{1: {1: 1},
 2: {2: 1},
 3: {3: 1},
 4: {2: 2},
 5: {5: 1},
 6: {2: 1, 3: 1},
 7: {7: 1},
 8: {2: 3},
 9: {3: 2},
 10: {2: 1, 5: 1},
 11: {11: 1},
 12: {2: 2, 3: 1},
 13: {13: 1},
 14: {2: 1, 7: 1},
 15: {3: 1, 5: 1},
 16: {2: 4},
 17: {17: 1},
 18: {2: 1, 3: 2},
 19: {19: 1}}

Given the factorizations, it is simple to detect squarefree integers.

In [14]:
def is_squarefree(factorization: Dict[int, int]) -> bool:
    """True if all exponents in the factorization are == 1; False otherwise."""
    return all(e == 1 for e in factorization.values())

for k, kfac in factorizations(20).items():
    if is_squarefree(kfac):
        print(f"{k} \t {'*'.join([f'{p}^{e}' for (p, e) in kfac.items()])}")

1 	 1^1
2 	 2^1
3 	 3^1
5 	 5^1
6 	 2^1*3^1
7 	 7^1
10 	 2^1*5^1
11 	 11^1
13 	 13^1
14 	 2^1*7^1
15 	 3^1*5^1
17 	 17^1
19 	 19^1


A function to count squarefree numbers $\leq n$ can be created using this idea.

In [15]:
def Q_factorizations(n: int) -> int:
    """Returns the number of squarefree positive intgers <= n"""
    return sum(is_squarefree(factorization) for factorization in factorizations(n + 1).values())

In [16]:
%time Q_factorizations(10)

CPU times: user 58 µs, sys: 3 µs, total: 61 µs
Wall time: 67 µs


7

In [17]:
%time Q_factorizations(10**3)

CPU times: user 5.47 ms, sys: 88 µs, total: 5.56 ms
Wall time: 7.08 ms


608

In [18]:
%time Q_factorizations(10**6)

CPU times: user 5.45 s, sys: 192 ms, total: 5.64 s
Wall time: 5.64 s


607926

Since our algorithm requires finding lots of prime factorizations, it has a high time and space complexity. This can be observed in the runtimes. However, this method is useful as a baseline and for testing other algorithms for the problem.

In [19]:
test_count = {0: 0}
for k, kfac in factorizations(10**3).items():
    test_count[k] = test_count[k - 1] + is_squarefree(kfac)

## Sieve method

We can use a sieve to find squarefree integers, similarly to how we can sieve to find primes.
Each number $\leq n$ that is not squarefree is divisible by $p^2$ for some prime $p \leq \sqrt{n}$.
For each prime $p \leq \sqrt{n}$, sieve the interval $1, 2, \ldots, n$ to remove all multiples of $p^2$.
The integers that remain after sieving are the squarefree ones.

In [27]:
def primes(n: int) -> List[int]:
    """Returns a list of primes <= n."""
    block = np.ones(n + 1, dtype=bool)
    block[0] = block[1] = False
    
    for p in range(2, integer_sqrt(n + 1) + 1):
        if block[p]:
            block[p*p::p] = False
    
    return np.where(block)[0].tolist()

In [28]:
timeit len(primes(10**6))

10.3 ms ± 33.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
# primes <= 30
primes(30)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

In [8]:
def Q_sieve(n: int) -> int:
    """Returns the number of squarefree integers <= n, using a sieve."""
    block = np.ones(n + 1, dtype=bool)
    block[0] = False
    bound = integer_sqrt(n) + 1
    
    for p in primes(bound):
        block[p*p::p*p] = False
    
    return np.count_nonzero(block)


for k in test_count:
    assert Q_sieve(k) == test_count[k]

NameError: name 'test_count' is not defined

In [137]:
%time Q_sieve(10**6)

CPU times: user 3.06 ms, sys: 505 µs, total: 3.57 ms
Wall time: 3.46 ms


607926

In [138]:
%time Q_sieve(10**8)

CPU times: user 120 ms, sys: 24.9 ms, total: 145 ms
Wall time: 144 ms


60792694

In [139]:
%time Q_sieve(10**9)

CPU times: user 1.13 s, sys: 405 ms, total: 1.54 s
Wall time: 1.54 s


607927124

This method is much faster, but still has $O(n)$ space since we store the entire interval $1 \ldots n$ in memory.
Time complexity is more than linear (has some logarithmic factors); i.e. $\tilde O(n)$ time complexity.

## Segmented Sieve

We can reduce the space complexity by using a segmented sieve.
The idea of a segmented sieve is to break the interval $1, 2, \ldots, n$ into smaller intervals of a given block size, and sieve each of these as we did in the previous algorithm.
This allows us to tune the space complexity by adjusting the block size at the cost of more bookkeeping.
We need to store the primes $\leq \sqrt{n}$, which requires space for $O(\sqrt{n} / \log(n))$ integers.
A segmented sieve with block size $B(n)$ would then give an $O(\sqrt{n}/log(n) + B(n))$ space requirement.

Breaking this up with a segmented sieve would not reduce the $\tilde O(n)$ time complexity, since we are still sieving all integers in the interval. However, we can reduce the time complexity via other methods.

In [140]:
def Q_segmented(n: int) -> int:
    prime_squares = [p**2 for p in primes(integer_sqrt(n) + 1)]
    block_size = 2**20
    count = 0
    
    for start in range(1, n + 1, block_size):
        block = np.ones(min(block_size, n - start + 1), dtype=bool)
        for p2 in prime_squares:
            # first multiple of p^2 that is >= start
            multiple = ((start // p2) + (start % p2 > 0))*p2
            block[multiple - start::p2] = False
        count += np.count_nonzero(block)
                
    return count


for k in test_count:
    assert Q_segmented(k) == test_count[k]

In [141]:
time Q_segmented(10**8)

CPU times: user 259 ms, sys: 0 ns, total: 259 ms
Wall time: 258 ms


60792694

In [142]:
time Q_segmented(10**9)

CPU times: user 5.49 s, sys: 0 ns, total: 5.49 s
Wall time: 5.49 s


607927124

## Combinatorial Method I

We first derive a recursive formula for $Q(x)$ by a combinatorial argument.

Partition all the $\lfloor x \rfloor$ positive integers $\leq x$ into subsets $S_k$ for $k = 1, 2, \ldots$ according to their largest square divisor $k^2$. The number of positive integers $\leq x$ having largest square divisor $k^2$ is then $|S_k| = Q(x / k^2)$. It follows that

$$
    \lfloor x \rfloor = \sum_{k \geq 1} |S_k| =  \sum_{1 \leq k \leq \sqrt{x}} Q\left(\frac{x}{k^2}\right).
$$

Rearranging gives a recursive formula for $Q(x)$

$$
    Q(x) = \lfloor x \rfloor - \sum_{2 \leq k \leq \sqrt{x}} Q \left( \frac{x}{k^2} \right).
$$

This can be used to implement a fast method to compute $Q(x)$ by storing previously computed values, a technique sometimes called *memoization* or just simply *caching* the recursion. Notice that we insert initial value $Q(1) = 1$ into the cache so that the recursion terminates.

In [143]:
def Q_memo(x, cache={1: 1}):
    """Returns number of squarefree integers <= x, using a memoized recursion."""
    if x not in cache:
        cache[x] = x - sum(Q_memo(x//k**2) for k in range(2, integer_sqrt(x) + 1))
    return cache[x]


for k in test_count:
    assert Q_memo(k) == test_count[k]

In [144]:
time Q_memo(10**6)

CPU times: user 5.56 ms, sys: 0 ns, total: 5.56 ms
Wall time: 4.6 ms


607926

In [145]:
time Q_memo(10**8)

CPU times: user 56.8 ms, sys: 0 ns, total: 56.8 ms
Wall time: 55.1 ms


60792694

In [146]:
time Q_memo(10**10)

CPU times: user 752 ms, sys: 0 ns, total: 752 ms
Wall time: 754 ms


6079270942

In [147]:
time Q_memo(10**12)

CPU times: user 9.12 s, sys: 0 ns, total: 9.12 s
Wall time: 9.12 s


607927102274

In [148]:
time Q_dp(10**12)

CPU times: user 9.43 s, sys: 0 ns, total: 9.43 s
Wall time: 9.43 s


607927102274

We can unravel the memoized recursion into an algorithm based on dynamic programming.

In [175]:
def Q_dp(x):
    if x in (0, 1):
        return x
    V = []
    k = 1
    while x//k**2 > x//(k + 1)**2:
        V.append(x//k**2)
        k += 1

    V += range(V[-1] - 1, -1, -1)
    cache = {i: i for i in V}
    
    for k in sorted(V):
        for l in range(2, integer_sqrt(k) + 1):
            cache[k] -= cache[k//l**2]
    
    return cache[x]


for k in test_count:
    assert Q_dp(k) == test_count[k]

In [182]:
def Q_moebius(n):
    bound = integer_sqrt(n) + 1
    block = np.array([n//d**2 for d in range(1, bound)])
    
    for p in primes(bound):
        block[p - 1::p] *= -1
        block[p*p - 1::p*p] = 0
    
    return np.sum(block)

In [183]:
time Q_moebius(10**12)

CPU times: user 2.16 s, sys: 0 ns, total: 2.16 s
Wall time: 2.16 s


607927102274

In [28]:
Q_moebius(10**6) == Q_dp(10**6) == Q_memo(10**6) == Q_sieve(10**6) == Q_segmented(10**6)

True